In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1358443/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_34_9,0.997649,0.801436,0.866795,0.989458,0.009866,1.327797,0.476468,0.015765,0.295148,0.099327,1.001376,0.103556,139.237339,218.464268,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_35_0,0.997647,0.801234,0.866342,0.992213,0.009875,1.329152,0.478089,0.010483,0.306251,0.099373,1.001377,0.103604,139.235487,218.462415,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_35_1,0.997639,0.801031,0.865883,0.991517,0.009908,1.330504,0.479728,0.011420,0.317280,0.099539,1.001382,0.103777,139.228824,218.455753,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_34_8,0.997634,0.801509,0.866854,0.989699,0.009930,1.327313,0.476256,0.015405,0.296949,0.099650,1.001385,0.103893,139.224354,218.451283,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_35_2,0.997626,0.800830,0.865420,0.990746,0.009964,1.331853,0.481385,0.012458,0.328230,0.099821,1.001390,0.104071,139.217500,218.444429,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,model_1_2_4,0.888952,0.658400,0.762575,0.707233,0.466044,2.284282,0.849257,0.217500,1.048428,0.682674,1.065004,0.711737,131.526953,210.753881,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
473,model_1_2_3,0.887297,0.656792,0.762889,0.718219,0.472990,2.295033,0.848133,0.209338,1.055557,0.687743,1.065972,0.717021,131.497362,210.724291,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
474,model_1_2_2,0.885592,0.655149,0.763155,0.728958,0.480148,2.306022,0.847180,0.201360,1.062780,0.692927,1.066971,0.722426,131.467322,210.694250,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
475,model_1_2_1,0.883834,0.653469,0.763373,0.739441,0.487522,2.317253,0.846401,0.193572,1.070099,0.698228,1.067999,0.727953,131.436840,210.663769,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
